In [1]:
!wget http://ai.stanford.edu/~acoates/stl10/stl10_binary.tar.gz -O stl10_binary.tar.gz

--2025-11-14 17:55:35--  http://ai.stanford.edu/~acoates/stl10/stl10_binary.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2640397119 (2.5G) [application/x-gzip]
Saving to: ‘stl10_binary.tar.gz’

stl10_binary.tar.gz 100%[===================>]   2.46G  18.6MB/s    in 3m 12s  

2025-11-14 17:58:47 (13.1 MB/s) - ‘stl10_binary.tar.gz’ saved [2640397119/2640397119]



In [2]:
import tarfile
import os

if not os.path.exists("stl10"):
    os.makedirs("stl10")

with tarfile.open("stl10_binary.tar.gz", "r:gz") as tar:
    tar.extractall(path="stl10/")

print("STL-10 dataset extracted!")


/tmp/ipython-input-3412765729.py:8: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path="stl10/")


STL-10 dataset extracted!


In [3]:
import numpy as np
from PIL import Image

def read_images(bin_file, out_dir):
    # Each image is 96x96x3 uint8
    with open(bin_file, 'rb') as f:
        data = np.fromfile(f, dtype=np.uint8)
        images = data.reshape(-1, 3, 96, 96).transpose(0,2,3,1)

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    for i, img in enumerate(images):
        im = Image.fromarray(img)
        im.save(os.path.join(out_dir, f"{i}.png"))

In [4]:
read_images("stl10/stl10_binary/train_X.bin", "stl10/images/train")
read_images("stl10/stl10_binary/test_X.bin", "stl10/images/test")

In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm
import pickle

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Combine train and test folders
image_folder_train = "stl10/images/train"
image_folder_test = "stl10/images/test"
image_files = [os.path.join(image_folder_train, f) for f in os.listdir(image_folder_train) if f.endswith(".png")]
image_files += [os.path.join(image_folder_test, f) for f in os.listdir(image_folder_test) if f.endswith(".png")]

embeddings = []
paths = []

for img_path in tqdm(image_files):
    image = Image.open(img_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        emb = model.get_image_features(**inputs)
    embeddings.append(emb.cpu().numpy()[0])
    paths.append(img_path)

# Step 5: Save embeddings and paths
os.makedirs("stl10/embeddings", exist_ok=True)
with open("stl10/embeddings/embeddings.pkl", "wb") as f:
    pickle.dump({"paths": paths, "embeddings": embeddings}, f)